# Dilation, translation, two-scale relationships and filters

The scaling and wavelet functions at successive scales are connected in two ways. One way is the translation and dilation, and the second one is the two-scale relationships. This is one of the key point of wavelet and multiwavelets.

In the previous exercise you have seen how the Haar scaling and wavelet functions at scale $n > 0$ can be obtained by translating and dilating the mother scaling and wavelet functions at scale $n=0$.

In general, for a multiwavelet basis you can always write the following:

$$
\phi^n_{jl}(x) = 2^{n/2} \phi^0_{j}(2^n x - l) \qquad \psi^n_{jl}(x) = 2^{n/2} \psi^0_{j}(2^n x - l)
$$

Alternatively, for two successive scales:
$$
\phi^{n+1}_{jl}(x) = 2^{1/2} \phi^n_{j}(2 x - l) \qquad \psi^{n+1}_{jl}(x) = 2^{1/2} \psi^n_{j}(2 x - l)
$$



The two-scale relationship instead connects the scaling functions at scale $n$ with the scaling and wavelet functions at scale $n-1$. In terms of linear algebra, this is a simple unitary transformation of the basis. borrowing from the original application of wavelets to signal processing, this unitary transformation is composed by four blocks called respectively $H_i$ and $G_i$ ($i=0,1$) where $H_i$ are the high-pass filters and $G_i$ are the low-pass filters. The whole transformation reads:

$$
\pmatrix{\boldsymbol{\phi}^n_{2l}(x) \\ \boldsymbol{\phi}^n_{2l+1}(x)} = 
\pmatrix{H_0 & G_0 \\ H_1 & G_1}
\pmatrix{\boldsymbol{\phi}^{n-1}_{l}(x) \\ \boldsymbol{\psi}^{n-1}_{l}(x)}
$$
where we have used **bold** symbols to the column vector of $k+1$ scaling or wavelet functions in a given node.

In the code cell below you will find a function which will return a selected filter ($H_i$ or $G_i$) for a given basis (Legendre/Interpolating), a given order (limited to $k \le 10$).

In [3]:
import sys, struct
import numpy as np

# Gets one of the four filter matrices
# Basis: L (legendre) or I (interpolating)
# Type: H or G
# Flag: 0 or 1
# n: from 1 to 11
def getfilter(basis = "L", ftype = "H", flag = 0, n = 4):
    order = n - 1
    name = "mwfilters/" + basis + "_" + ftype + "0_" + str(order)
    filterFile = open(name, 'rb')
    filterData = filterFile.read()
    filterList = []
    for i in range((n)*(n)):
        rstart = i * 8
        rend = (i + 1) * 8
        filterList.append(struct.unpack_from("d",filterData[rstart:rend]))
    filterArray = np.array(filterList)
    filterMatrix = np.ndarray((n, n),buffer=filterArray)
    if (flag == 1):
        if (ftype == "G" and n%2 != 0 ):
            sign_np = -1
        else:
            sign_np = 1
        for i in range(n):
            for j in range(n):
                sign_ij = 1
                if (((i+j)%2) == 0):
                    sign_ij = -1
                sign = sign_ij * sign_np
                filterMatrix[i,j] *= sign
    return filterMatrix

#filters for a third order (k+1=4) Legendre basis

basis = "L" # alternative "I" for interpolating
n = 4 # n = k+1  (avaliable up to k=10)
H0 = getfilter(basis, "H", 0, n)
H1 = getfilter(basis, "H", 1, n)
G0 = getfilter(basis, "G", 0, n)
G1 = getfilter(basis, "G", 1, n)
print(H0)
print(H1)
print(G0)
print(G1)

[[ 7.07106781e-01  0.00000000e+00  4.88535468e-77  0.00000000e+00]
 [-6.12372436e-01  3.53553391e-01 -4.48460293e-77  2.23275976e-77]
 [ 9.16004002e-78 -6.84653197e-01  1.76776695e-01 -3.96935068e-77]
 [ 2.33853587e-01  4.05046294e-01 -5.22912517e-01  8.83883476e-02]]
[[-7.07106781e-01  0.00000000e+00 -4.88535468e-77  0.00000000e+00]
 [-6.12372436e-01 -3.53553391e-01 -4.48460293e-77 -2.23275976e-77]
 [-9.16004002e-78 -6.84653197e-01 -1.76776695e-01 -3.96935068e-77]
 [ 2.33853587e-01 -4.05046294e-01 -5.22912517e-01 -8.83883476e-02]]
[[ 6.54942862e-76 -1.53392998e-01 -5.94088526e-01  3.51467512e-01]
 [-1.54303350e-01 -2.67261242e-01 -1.72516390e-01  6.12372436e-01]
 [-1.98162199e-75 -8.78668779e-02 -3.40306955e-01 -6.13571991e-01]
 [ 2.15645487e-01  3.73508940e-01  4.43622131e-01  3.42326598e-01]]
[[-6.54942862e-76 -1.53392998e-01  5.94088526e-01  3.51467512e-01]
 [-1.54303350e-01  2.67261242e-01 -1.72516390e-01 -6.12372436e-01]
 [ 1.98162199e-75 -8.78668779e-02  3.40306955e-01 -6.135719

## Exercises

- Show (numerically) that the four filter matrices constitute a **unitary transformation**.
- Use the dilation and translation relationships to obtain, for a given scaling/wavelet function, the corresponding function at the next **finer scale** (larger $n$).
- Use the two-scale relationship to verify that the scaling function obtained by translation/dilation can also be obtained by linear transformation of the scaling&wavelet functions at the next **coraser scale** (smaller $n$).

In [1]:
from vampyr import LegendreBasis, InterpolatingBasis
import matplotlib.pyplot as plt
import numpy as np

# Create Legendre basis of order 4 and Interpolating basis of order 3
legendre_4 = LegendreBasis(order=4)
interpol_3 = InterpolatingBasis(order=3)